In [118]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [173]:
import plotly.graph_objects as go
def plot_predicted_vs_actual(y_actual, y_predicted, val_or_test):
    # Create the figure
    fig = go.Figure()

    # Add the line for the actual values
    fig.add_trace(go.Scatter(
        x=list(range(len(y_actual))),
        y=y_actual,
        mode='lines',
        name= 'Valores reais'
    ))

    # Add the line for the predicted values
    fig.add_trace(go.Scatter(
        x=list(range(len(y_predicted))),
        y=y_predicted,
        mode='lines',
        name= 'Valores preditos'
    ))

    # Configure layout
    fig.update_layout(
        title= 'Valores Preditos vs Valores Reais ' + val_or_test,
        xaxis=dict(title='Amostra'),
        yaxis=dict(title='Valor')
    )

    fig.show()

# Selecting training and testing data

In [133]:
# Define the list of unique identifiers
ids = df['file'].unique()
 
# Separate the test data for each identifier.
test_data = []
for id_ in ids:
    # Filter the data only for the current identifier
    data_id = df[df['file'] == id_]
    # Split 20% of the data for testing
    data_id_train, data_id_test = train_test_split(data_id, test_size=0.2, random_state=13, shuffle=False)
    # Add the test data to the general list of test data
    test_data.append(data_id_test)

# Concatenate the test data into a single DataFrame
test_data = pd.concat(test_data)

# Obtain the training data by removing the test data
train_data = df.drop(test_data.index)

# Check the shape of the training and test data
print("Shape dos dados de treinamento:", train_data.shape)
print("Shape dos dados de test:", test_data.shape)

Shape dos dados de treinamento: (60413, 19)
Shape dos dados de test: (15114, 19)


In [134]:
# Divida as variáveis independentes e a variável dependente
X_train = train_data[['Entrada']]
y_train = train_data['Saida']

X_test = test_data[['Entrada']]
y_test = test_data['Saida']

In [19]:
# Divida as variáveis independentes e a variável dependente
X_train = train_data[['Entrada independente']]
y_train = train_data['Saida']

X_test = test_data[['Entrada independente']]
y_test = test_data['Saida']

In [131]:
# Definir o modelo base
model = xgb.XGBRegressor()

# Definir o dicionário de parâmetros para a busca em grade
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.5, 0.3, 0.1],
    'max_depth': [3, 6, 9]
}

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)

# Treinar o objeto GridSearchCV
grid_search.fit(X_train, y_train)

# Obter os melhores parâmetros encontrados
best_params = grid_search.best_params_
print(best_params)


{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}


# Building and training the model

In [143]:
# Criando Modelo XGBoost

# model = xgb.XGBRegressor(n_estimators= 100, learning_rate=0.1, max_depth= 6)
model = xgb.XGBRegressor(n_estimators= 100, learning_rate=0.3, max_depth=3)

# Treinando Modelo
model.fit(X_train, y_train)

# Faça previsões sobre os dados de teste
y_pred = model.predict(X_test)

# Calcule o erro quadrático médio (MSE) nas previsões
mse = mean_squared_error(y_test, y_pred)
print("Erro médio quadrado (MSE):", mse)

r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)

Erro médio quadrado (MSE): 86.4469849735229
R-squared: 0.5973242790982266


## Feature Importance

In [ ]:
# Obter importancia de Features
feature_importance = model.feature_importances_

# Obtenha os nomes dos recursos
nomes_features = X_train.columns

# Criando um DataFrame com as importâncias e nomes dos recursos
df_importance = pd.DataFrame({'Feature': nomes_features, 'Importance': feature_importance})

# Criando um DataFrame com as importâncias e nomes dos recursos
df_importance = df_importance.sort_values('Importance', ascending=False).reset_index(drop=True)

# Plot
plt.figure(figsize=(10, 6))
plt.barh(df_importance['Feature'], df_importance['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

## Real vs Prediction

In [ ]:
df_test = X_test.copy()

df_test['y_test'] = y_test
df_test['pred'] = y_pred

df_test = df_test.reset_index(drop=True)

df_test

# Plotting Predicted vs Real

In [175]:
plot_predicted_vs_actual(df_test['y_test'], df_test['pred'], '(Dados de Teste, shuffle=False, Primeira abordagem)')